In [155]:
import numpy as np
import pandas as pd

In [39]:
#1.1
modules_mass = pd.read_fwf("./modules_mass.txt", header=None)

In [52]:
def get_modules_fuel(modules_mass):
    total_fuel = 0
    for mass in modules_mass[0]:
        total_fuel += int(mass/3) -2
    return total_fuel
get_modules_fuel(modules_mass)

3374289

In [58]:
#1.2
def get_total_fuel(modules_mass):
    total_fuel = 0
    for mass in modules_mass[0]:
        module_fuel = int(mass/3) - 2
        total_fuel += module_fuel + get_fuel_of_fuel(module_fuel)
    return total_fuel

def get_fuel_of_fuel(fuel):
    new_fuel = int(fuel/3) - 2
    if new_fuel <= 0: return 0
    return new_fuel + fuel_of_fuel(new_fuel)
get_total_fuel(modules_mass)

In [192]:
#2.1
grav_assist_prog = np.loadtxt("./grav_assist_prog.txt", delimiter=",").astype(int)
grav_assist_prog[1] = 12
grav_assist_prog[2] = 2

In [ ]:
def computer(code):
    for i in range( 0, len(code), 4 ):
        optcode = code[i]
        a = code[i+1]
        b = code[i+2]
        c = code[i+3]
        if optcode == 99: 
            return code
        elif optcode == 1:
            code[c] = code[a] + code[b]
        elif optcode == 2:
            code[c] = code[a] * code[b]
        else: return "Unknown optcode"
computer( grav_assist_prog.copy() )

In [186]:
#2.2
def find_noun_verb():
    for i in range(0,100):
        for j in range(0,100):
            grav_assist_prog_copy = grav_assist_prog.copy()
            grav_assist_prog_copy[1] = i
            grav_assist_prog_copy[2] = j
            output = computer(grav_assist_prog_copy)[0]
            if output == 19690720: 
                return i,j
    return False
find_noun_verb()

In [320]:
#3.1
paths = []
with open("crossed_wires.txt") as f:
    paths = f.read().splitlines()

m = {'U': (0,1),
     'D': (0,-1),
     'L': (-1,0),
     'R': (1,0)}

def get_path_range(path):
    x = y = l = r = u = d = 0
    for i in range( len(path) ):
        direction, val = ( path[i][0], int(path[i][1:]) )
        x = x + m[direction][0]*val
        y = y + m[direction][1]*val
        if x > r: r=x 
        if x < l: l = x 
        if y > u: u=y 
        if y < d: d = y 
    return (r,l), (u,d)

def get_closest_intersection_dist(wire):
    w1 = get_wire_path(wire[0])[1:]
    w2 = get_wire_path(wire[1])[1:]
    intersections = get_intersections(w1, w2)
    dist = lambda pt: abs(pt[0]) + abs(pt[1])
    closest_intr = min(intersections, key = lambda y: dist(y))
    return dist(closest_intr)

def get_wire_path(path):
    path = path.split(',')
    all_steps = [[0,0]]
    x_range, y_range = get_path_range(path)
    
    for i in range( len(path) ):
        step = ( path[i][0], int(path[i][1:]) )
        all_steps = update_wire_coordinates(step, all_steps, x_range, y_range)
    return all_steps

def update_wire_coordinates(step, coordinates, x_range, y_range):
    d, u = step
    x, y = coordinates[-1]
    for i in range(1, u+1):
        x = x + m[d][0]
        y = y + m[d][1]
        if (not [x,y] in coordinates) and in_bounds(x, y, x_range, y_range): 
            coordinates.append([x,y])
    return coordinates

def in_bounds(x, y, x_range, y_range):
    return (x <= x_range[0] and x >= x_range[1]) and (y <= y_range[0] and y >= y_range[1])
    
    
def get_intersections(steps1, steps2):
    intercections = []
    for step1 in steps1:
        for step2 in steps2:
            if step1 == step2: intercections.append(step1)
    return intercections

In [321]:
get_closest_intersection_dist(["R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51","U98,R91,D20,R16,D67,R40,U7,R15,U6,R7"])

135